<a href="https://github.com/seth-mc/sethsgit"><img src= "https://i.ibb.co/WtQWBMc/sethlogo.png" alt="sethlogo" border="0" width = 150 align = left></a>

## Capstone Maps Assignment


### Imports

In [ ]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge lxml --yes
import lxml

!conda install -c conda-forge bs4 --yes
import bs4

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## Q1

### Dataframe Build

In [ ]:
# scraping the WIKI
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)  # reads wiki table
df = df[0]  # converts the list of dfs to a dataframe
df = df[df['Borough'] != 'Not assigned']  # getting rid of 'not assigned' borough columns.
df.reset_index(drop=True, inplace=True)

df.head()

In [ ]:
df.shape

## Q2

### Appending Coordinates to Dataframe

In [ ]:
# geospatial dataframe creation
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)
geospatial_data.head()

In [ ]:
# merging the two
merged_data = pd.merge(df, geospatial_data, on='Postal Code')
merged_data.head()

## Q3

### Exploring and Clustering the Neighborhoods in Toronto

In [ ]:
# creating a new row for coordinates
merged_data['Coordinates'] = list(zip(merged_data['Latitude'], merged_data['Longitude']))
merged_data.head()

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Borough'], merged_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#449e56',
        fill=True,
        fill_color='#abffbc',
        fill_opacity=0.7,
        tiles="Stamen Toner",
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
# map does not show on Git due to trust, but a bunch of green circles are plotted in toronto,
# each with info about Toronto from the dataset.